# CIFAR-10 CNN Training with PyTorch - Self-Contained Notebook

This notebook contains all the code for training a CIFAR-10 CNN model with advanced features:
- C1C2C3C4 architecture with Depthwise Separable and Dilated Convolutions
- Albumentations data augmentation
- Global Average Pooling (GAP)
- Receptive field > 44 pixels
- < 200k parameters

**All code is embedded in this single notebook - no external files needed!**

In [1]:
# Install required packages
!pip install torch torchvision torchsummary numpy matplotlib albumentations tqdm

In [2]:
# ================================
# logger_setup.py - Logging Setup
# ================================
import logging
import os

_LOGGING_INITIALIZED = False

class TqdmLoggingHandler(logging.StreamHandler):
    """A logging handler that plays nicely with tqdm progress bars."""
    def emit(self, record):
        try:
            from tqdm import tqdm  # Lazy import so tqdm isn't a hard dependency
            msg = self.format(record)
            tqdm.write(msg)
        except Exception:
            # Fallback to normal stream behavior
            super().emit(record)

def setup_logging(log_to_file=False, log_dir='logs'):
    """Set up simple logging configuration.

    Args:
        log_to_file (bool): If True, also log to a file (default: False)
        log_dir (str): Directory for log files if log_to_file is True
    """
    global _LOGGING_INITIALIZED

    # If already initialized, don't recreate handlers (prevents truncation mid-run)
    if _LOGGING_INITIALIZED:
        logger = logging.getLogger()
        # If file logging is requested and not present yet, add it (append mode)
        if log_to_file and not any(isinstance(h, logging.FileHandler) for h in logger.handlers):
            try:
                if not os.path.exists(log_dir):
                    os.makedirs(log_dir, exist_ok=True)
                log_file = os.path.join(log_dir, 'training.log')
                file_handler = logging.FileHandler(log_file, mode='a', encoding='utf-8')
                file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
                file_handler.setLevel(logging.INFO)
                logger.addHandler(file_handler)
            except Exception:
                pass
        return logger

    # Configure basic logging format
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    # Set up console handler
    console_handler = TqdmLoggingHandler()
    console_handler.setFormatter(formatter)
    console_handler.setLevel(logging.INFO)

    # Configure root logger
    root_logger = logging.getLogger()
    root_logger.setLevel(logging.INFO)

    # Remove any existing handlers to avoid duplicates
    for handler in root_logger.handlers[:]:
        root_logger.removeHandler(handler)

    # Add console handler
    root_logger.addHandler(console_handler)

    # Add file handler if requested
    if log_to_file:
        try:
            if not os.path.exists(log_dir):
                os.makedirs(log_dir, exist_ok=True)
            log_file = os.path.join(log_dir, 'training.log')
            # Truncate once at initial setup to start fresh, then use append mode
            try:
                with open(log_file, 'w', encoding='utf-8'):
                    pass
            except Exception:
                # If truncation fails, proceed; handler will create/append
                pass
            file_handler = logging.FileHandler(log_file, mode='a', encoding='utf-8')
            file_handler.setFormatter(formatter)
            file_handler.setLevel(logging.INFO)
            root_logger.addHandler(file_handler)
        except Exception:
            # Silently continue with console-only logging if file logging fails
            pass

    _LOGGING_INITIALIZED = True
    return root_logger

In [3]:
# ================================
# data_setup.py - Data Loading & Augmentation
# ================================
import torch.utils as utils
from torchvision import datasets
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

class AlbumentationsTransform:
    """Wrapper to make Albumentations compatible with PyTorch datasets"""
    def __init__(self, transform):
        self.transform = transform

    def __call__(self, img):
        # Convert PIL to numpy
        img = np.array(img)
        # Apply albumentations transform
        transformed = self.transform(image=img)
        return transformed['image']

class DataSetup:
    def __init__(self, batch_size_train=64, batch_size_test=1000, shuffle_train=True, shuffle_test=False, num_workers=2, pin_memory=None, train_transforms=None, test_transforms=None):
        self.batch_size_train = batch_size_train
        self.batch_size_test = batch_size_test
        self.shuffle_train = shuffle_train
        self.shuffle_test = shuffle_test
        self.num_workers = num_workers
        self.pin_memory = pin_memory
        self.train_transforms = train_transforms if train_transforms else self.get_train_transforms()
        self.test_transforms = test_transforms if test_transforms else self.get_test_transforms()
        self.train_loader = self.get_train_loader()
        self.test_loader = self.get_test_loader()

    def get_train_transforms(self):
        """Albumentations transforms for training with required augmentations"""
        # CIFAR-10 mean: (0.4914, 0.4822, 0.4465) -> [125, 123, 114] for 0-255 range
        fill_value = [125, 123, 114]

        train_transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(
                shift_limit=0.1,
                scale_limit=0.1,
                rotate_limit=15,
                p=0.5
            ),
            A.CoarseDropout(
                max_holes=1,
                max_height=16,
                max_width=16,
                min_holes=1,
                min_height=16,
                min_width=16,
                fill_value=fill_value,
                mask_fill_value=None,
                p=0.5
            ),
            A.Normalize(
                mean=(0.4914, 0.4822, 0.4465),
                std=(0.2470, 0.2435, 0.2616)
            ),
            ToTensorV2()
        ])
        return AlbumentationsTransform(train_transform)

    def get_test_transforms(self):
        """Albumentations transforms for testing (only normalization)"""
        test_transform = A.Compose([
            A.Normalize(
                mean=(0.4914, 0.4822, 0.4465),
                std=(0.2470, 0.2435, 0.2616)
            ),
            ToTensorV2()
        ])
        return AlbumentationsTransform(test_transform)

    def get_train_datasets(self):
        return datasets.CIFAR10('../data', train=True, download=True, transform=self.train_transforms)

    def get_test_datasets(self):
        return datasets.CIFAR10('../data', train=False, download=True, transform=self.test_transforms)

    def get_train_loader(self):
        train_dataset = self.get_train_datasets()
        return utils.data.DataLoader(train_dataset, batch_size=self.batch_size_train, shuffle=self.shuffle_train, num_workers=self.num_workers, pin_memory=self.pin_memory)

    def get_test_loader(self):
        test_dataset = self.get_test_datasets()
        return utils.data.DataLoader(test_dataset, batch_size=self.batch_size_test, shuffle=self.shuffle_test, num_workers=self.num_workers, pin_memory=self.pin_memory)

In [4]:
# ================================
# cifar10model_v0.py - Model Architecture
# ================================
import torch
import torch.nn as nn
import torch.nn.functional as F

class DepthwiseSeparableConv(nn.Module):
    """Depthwise Separable Convolution = Depthwise + Pointwise"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(DepthwiseSeparableConv, self).__init__()

        # Depthwise: each input channel convolved separately
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size,
                                 stride=stride, padding=padding, groups=in_channels, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)

        # Pointwise: 1x1 conv to mix channels
        self.pointwise = nn.Conv2d(in_channels, out_channels, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        x = F.relu(self.bn1(self.depthwise(x)))
        x = F.relu(self.bn2(self.pointwise(x)))
        return x

class Net(nn.Module):
    """CIFAR-10 CNN with C1C2C3C4 architecture, Depthwise Sep Conv, Dilated Conv, and GAP."""
    def __init__(self):
        super(Net, self).__init__()

        # C1: Initial feature extraction (32x32 -> 32x32)
        self.c1 = nn.Sequential(
            nn.Conv2d(3, 8, 3, padding=1, bias=False),    # 32x32x8, RF=3
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Conv2d(8, 16, 3, padding=1, bias=False),   # 32x32x16, RF=5
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Conv2d(16, 32, 3, padding=1, bias=False),  # 32x32x32, RF=7
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.05),
        )

        # C2: Feature extraction with Dilated Convolutions (32x32 -> 32x32)
        self.c2 = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding=0, dilation=1, bias=False),  # 32x32x32, RF=9
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Conv2d(32, 32, 3, padding=2, dilation=2, bias=False),  # 32x32x32, RF=13
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Conv2d(32, 32, 3, padding=0, dilation=1, bias=False),  # 32x32x32, RF=21
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.05),
        )

        # C3: Pattern recognition with Depthwise Separable Conv (32x32 -> 16x16)
        self.c3 = nn.Sequential(
            nn.Conv2d(32, 32, 3, padding=1, stride=2),  # 32x32x32 -> 16x16x32, RF=23
            nn.BatchNorm2d(32),
            nn.ReLU(),
            DepthwiseSeparableConv(32, 64, kernel_size=3, stride=1, padding=1),  # 16x16x64, RF=25
            nn.Conv2d(64, 64, 3, padding=1, bias=False),  # 16x16x64, RF=27
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.05),

        )

        # C4: Final convolution with stride=2 (16x16 -> 8x8), then GAP + FC
        self.c4 = nn.Sequential(
            nn.Conv2d(64, 128, 3, stride=1, padding=1, bias=False),  # 8x8x128, RF=55
            #nn.BatchNorm2d(128),
            #nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),  # 1x1x128, RF=covers entire input
            nn.Flatten(),
            nn.Linear(128, 10)  # FC after GAP to target classes
        )

    def forward(self, x):
        x = self.c1(x)
        x = self.c2(x)
        x = self.c3(x)
        x = self.c4(x)
        return F.log_softmax(x, dim=1)

class set_config_v0:
    """Basic configuration for CIFAR-10 training."""
    def __init__(self):
        self.epochs = 35
        self.nll_loss = torch.nn.NLLLoss()
        self.criterion = self.nll_loss

    def setup(self, model, use_onecycle: bool = True):
        self.use_onecycle = use_onecycle
        base_lr = 0.01
        self.optimizer = torch.optim.SGD(model.parameters(), lr=base_lr, momentum=0.9)
        self.device = next(model.parameters()).device
        self.dataloader_args = self.get_dataloader_args()
        self.data_setup_instance = DataSetup(**self.dataloader_args)
        if self.use_onecycle:
            steps_per_epoch = len(self.data_setup_instance.train_loader)
            self.scheduler = torch.optim.lr_scheduler.OneCycleLR(
                self.optimizer,
                max_lr=base_lr,
                epochs=self.epochs,
                steps_per_epoch=steps_per_epoch,
                pct_start=0.2,
                div_factor=10,
                final_div_factor=100,
                anneal_strategy='cos'
            )
            self.scheduler.batch_step = True
            logging.getLogger().info(
                f"Model v0: OneCycleLR max_lr={base_lr} pct_start=0.2 div_factor=10 final_div_factor=100 epochs={self.epochs}"
            )
        else:
            self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=6, gamma=0.1)
            self.scheduler.batch_step = False
            logging.getLogger().info(
                f"Model v0: StepLR lr={base_lr} step_size=6 gamma=0.1"
            )
        logging.getLogger().info(f"Dataloader arguments: {self.dataloader_args}")
        return self

    def get_dataloader_args(self):
        if hasattr(self, 'device') and self.device.type == "cuda":
            args = dict(batch_size_train=32, batch_size_test=1000, shuffle_train=True, shuffle_test=False,
                        num_workers=2, pin_memory=True)
        else:
            args = dict(batch_size_train=32, batch_size_test=1000, shuffle_train=True, shuffle_test=False)
        logging.info(f"Model v0 dataloader args: {args}")
        return args

In [5]:
# ================================
# train_test.py - Training & Testing Logic
# ================================
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import logging

class train_test_model:

    def __init__(self, model, device, train_loader, test_loader,criterion,optimizer,scheduler,epochs=1):
        self.model = model
        self.device = device
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.train_acc_list = []
        self.test_acc_list = []
        self.F = F  # Assign torch.nn.functional to self.F for easier access
        self.epochs = epochs

    def train(self, model, device, train_loader, optimizer, criterion,epoch):
        self.model.train()
        pbar = tqdm(self.train_loader, desc="Training", leave=True)
        train_loss, correct, processed = 0, 0, 0
        # Train with progress bar

        for batch_idx, (data, target) in enumerate(pbar, 1):
            # get samples and move to device
            data, target = data.to(self.device), target.to(self.device)
            # Initialize optimizer
            self.optimizer.zero_grad()
            # Prediction
            output = self.model(data)
            # Calculate loss
            loss = self.F.nll_loss(output, target)
            # Backpropagation
            loss.backward()
            # Gradient clipping to stabilize higher LR OneCycle swings
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=2.0)
            self.optimizer.step()
            # Per-batch scheduler stepping (e.g., OneCycleLR) if attribute present
            if hasattr(self, 'scheduler') and getattr(self.scheduler, 'batch_step', False):
                self.scheduler.step()
            # -----------------------------
            # Accumulate loss and calculate accuracy
            train_loss += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            batch_correct = pred.eq(target.view_as(pred)).sum().item()
            correct += batch_correct
            processed += len(data)

            # Calculate current metrics
            current_loss = train_loss / processed
            current_accuracy = 100. * correct / processed

            # Update progress bar only (add LR peek occasionally)
            if hasattr(self, 'scheduler') and getattr(self.scheduler, 'batch_step', False):
                current_lr = self.scheduler.get_last_lr()[0]
                status = f"Train Loss={current_loss:.4f} Acc={current_accuracy:.2f}% LR={current_lr:.4f}"
            else:
                status = f"Train Loss={current_loss:.4f} Accuracy={current_accuracy:.2f}%"
            pbar.set_description(desc=status)

        # Final epoch-level logging for training metrics
        epoch_accuracy = 100. * correct / len(self.train_loader.dataset)
        logging.info(
            f'Epoch {epoch:02d}/{self.epochs}: Train set final results: Average loss: {train_loss:.4f}, '
            f'Accuracy: {correct}/{len(self.train_loader.dataset)} ({epoch_accuracy:.2f}%)'
        )
        return epoch_accuracy

    def test(self, model, device, test_loader, criterion,epoch):
        self.model.eval()
        test_loss, correct = 0, 0
        # Test with progress bar

        with torch.no_grad():
            pbar = tqdm(self.test_loader, desc="Testing", leave=True)
            for batch_idx, (data, target) in enumerate(pbar, 1):
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                test_loss += self.F.nll_loss(output, target, reduction='sum').item()
                pred = output.argmax(dim=1, keepdim=True)
                batch_correct = pred.eq(target.view_as(pred)).sum().item()
                correct += batch_correct

                # Calculate current metrics
                current_loss = test_loss / (batch_idx * len(data))
                current_accuracy = 100. * correct / (batch_idx * len(data))

                # Update progress bar only
                status = f"Test Loss={current_loss:.4f} Accuracy={current_accuracy:.2f}%"
                pbar.set_description(desc=status)

        test_loss /= len(self.test_loader.dataset)
        acc = 100. * correct / len(self.test_loader.dataset)
        logging.info(f'Epoch {epoch:02d}/{self.epochs}:Test set final results: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(self.test_loader.dataset)} ({acc:.2f}%)')
        return acc

    def do_training(self,epoch):
        return self.train(self.model, self.device, self.train_loader, self.optimizer, self.criterion,epoch)

    def do_testing(self,epoch):
        return self.test(self.model, self.device, self.test_loader, self.criterion,epoch)

    def run_epoch(self):
        logging.info(f"Training model for {self.epochs} epochs")
        for epoch in range(1, self.epochs+1):
            train_acc = self.do_training(epoch=epoch)
            test_acc = self.do_testing(epoch=epoch)
            # Epoch-level scheduler step only if not using per-batch scheduler
            if hasattr(self.scheduler, 'batch_step') and not getattr(self.scheduler, 'batch_step'):
                self.scheduler.step()
            elif not hasattr(self.scheduler, 'batch_step'):
                # Legacy schedulers
                try:
                    self.scheduler.step()
                except Exception:
                    pass
            self.train_acc_list.append(train_acc)
            self.test_acc_list.append(test_acc)

            #logging.info(f"Epoch {epoch:02d}/{self.epochs}: Train={train_acc:.2f}%, Test={test_acc:.2f}%, LR={self.scheduler.get_last_lr()[0]:.6f}")

    def plot_results(self):
        plt.plot(self.train_acc_list, label='Train Acc')
        plt.plot(self.test_acc_list, label='Test Acc')
        plt.legend()
        plt.title("Training vs Test Accuracy")
        plt.show()

In [6]:
# ================================
# summarizer.py - Model Summary & Checks
# ================================
import torch.nn as nn
from torchsummary import summary

# -----------------------------
# 8. Model Architecture Checks
# -----------------------------
def model_checks(model):
    import logging

    logging.info('--- Model Architecture Checks ---')
    # Total Parameter Count
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logging.info(f'Total Parameters: {total_params:,}')
    logging.info(f'Trainable Parameters: {trainable_params:,}\n')

    logging.info('Layer-wise Parameter Details (in model order):')
    logging.info('-'*100)

    def get_layer_details(module):
        details = ''
        if isinstance(module, nn.Conv2d):
            details = (f'Convolution: {module.in_channels}->{module.out_channels} channels, '
                      f'kernel {module.kernel_size}, stride {module.stride}, padding {module.padding}, '
                      f'groups {module.groups}, bias {module.bias is not None}')
        elif isinstance(module, nn.BatchNorm2d):
            details = f'BatchNorm: {module.num_features} features, eps={module.eps}, momentum={module.momentum}'
        elif isinstance(module, nn.ReLU):
            details = 'Activation: ReLU'
        elif isinstance(module, nn.ReLU6):
            details = 'Activation: ReLU6'
        elif isinstance(module, nn.LeakyReLU):
            details = f'Activation: LeakyReLU (negative_slope={module.negative_slope})'
        elif isinstance(module, nn.MaxPool2d):
            details = f'MaxPool: kernel {module.kernel_size}, stride {module.stride}, padding {module.padding}'
        elif isinstance(module, nn.AvgPool2d):
            details = f'AvgPool: kernel {module.kernel_size}, stride {module.stride}, padding {module.padding}'
        elif isinstance(module, nn.AdaptiveAvgPool2d):
            details = f'AdaptiveAvgPool: output size {module.output_size}'
        elif isinstance(module, nn.Dropout):
            details = f'Dropout: probability {module.p}'
        elif isinstance(module, nn.Dropout2d):
            details = f'Dropout2d: probability {module.p}'
        elif isinstance(module, nn.Linear):
            details = f'Linear: {module.in_features}->{module.out_features}, bias {module.bias is not None}'
        elif isinstance(module, nn.Flatten):
            details = 'Flatten'
        return details

    # Get layers in order as defined in model
    for name, module in model.named_children():
        if isinstance(module, nn.Sequential):
            logging.info(f"\nBlock: {name} (Sequential)")
            for subname, submodule in module.named_children():
                layer_name = f'{name}.{subname} ({submodule.__class__.__name__})'
                layer_params = sum(p.numel() for p in submodule.parameters())
                details = get_layer_details(submodule)
                logging.info(f'  {layer_name:50} | Params: {layer_params:6,d} | {details}')
        else:
            layer_name = f'{name} ({module.__class__.__name__})'
            layer_params = sum(p.numel() for p in module.parameters())
            details = get_layer_details(module)
            logging.info(f'  {layer_name:50} | Params: {layer_params:6,d} | {details}')

    logging.info('-'*100)
    logging.info('\nLayer Type Summary:')

    # Count all layer types
    layer_types = {
        'Conv2d': [m for m in model.modules() if isinstance(m, nn.Conv2d)],
        'BatchNorm2d': [m for m in model.modules() if isinstance(m, nn.BatchNorm2d)],
        'ReLU': [m for m in model.modules() if isinstance(m, (nn.ReLU, nn.ReLU6))],
        'LeakyReLU': [m for m in model.modules() if isinstance(m, nn.LeakyReLU)],
        'MaxPool2d': [m for m in model.modules() if isinstance(m, nn.MaxPool2d)],
        'AvgPool2d': [m for m in model.modules() if isinstance(m, nn.AvgPool2d)],
        'AdaptiveAvgPool2d': [m for m in model.modules() if isinstance(m, nn.AdaptiveAvgPool2d)],
        'Dropout': [m for m in model.modules() if isinstance(m, nn.Dropout)],
        'Dropout2d': [m for m in model.modules() if isinstance(m, nn.Dropout2d)],
        'Linear': [m for m in model.modules() if isinstance(m, nn.Linear)],
        'Flatten': [m for m in model.modules() if isinstance(m, nn.Flatten)]
    }

    for layer_type, layers in layer_types.items():
        if layers:  # Only show if there are layers of this type
            logging.info(f'{layer_type:20} layers used: {len(layers):3d}')

    logging.info('-'*100)

In [7]:
# ================================
# receptive_field_calculator.py - RF Calculation
# ================================
def calculate_receptive_field(model):
    """
    Calculate receptive field for each layer in the model.
    Formula: RF_new = RF_previous + (kernel_size - 1) * dilation for stride=1
             RF_new = RF_previous * stride + (kernel_size - 1) * dilation for stride>1
    """
    rf = 1  # Starting receptive field (single pixel)
    print("Layer-by-layer receptive field calculation:")
    print(f"Input: RF = {rf}")

    # C1 block
    print("\nC1 Block:")
    # Conv2d(3, 8, 3, padding=1)
    rf = rf + (3 - 1) * 1
    print(f"Conv2d(3->8, 3x3, p=1): RF = {rf}")

    # Conv2d(8, 16, 3, padding=1)
    rf = rf + (3 - 1) * 1
    print(f"Conv2d(8->16, 3x3, p=1): RF = {rf}")

    # Conv2d(16, 32, 3, padding=1)
    rf = rf + (3 - 1) * 1
    print(f"Conv2d(16->32, 3x3, p=1): RF = {rf}")

    # C2 block
    print("\nC2 Block:")
    # Conv2d(32, 32, 3, padding=1, dilation=1)
    rf = rf + (3 - 1) * 1
    print(f"Conv2d(32->32, 3x3, p=1, d=1): RF = {rf}")

    # Conv2d(32, 32, 3, padding=2, dilation=2)
    rf = rf + (3 - 1) * 2
    print(f"Conv2d(32->32, 3x3, p=2, d=2): RF = {rf}")

    # Conv2d(32, 32, 3, padding=4, dilation=4)
    rf = rf + (3 - 1) * 4
    print(f"Conv2d(32->32, 3x3, p=4, d=4): RF = {rf}")

    # C3 block
    print("\nC3 Block:")
    # DepthwiseSeparableConv depthwise: Conv2d(32, 32, 3, stride=2, padding=1, groups=32)
    rf = rf * 2 + (3 - 1) * 1
    print(f"DepthwiseConv(32->32, 3x3, s=2, p=1): RF = {rf}")

    # Pointwise doesn't change RF
    print(f"PointwiseConv(32->64, 1x1): RF = {rf} (unchanged)")

    # Conv2d(64, 64, 3, padding=1)
    rf = rf + (3 - 1) * 1
    print(f"Conv2d(64->64, 3x3, p=1): RF = {rf}")

    # C4 block
    print("\nC4 Block:")
    # Conv2d(64, 128, 3, stride=2, padding=1)
    rf = rf * 2 + (3 - 1) * 1
    print(f"Conv2d(64->128, 3x3, s=2, p=1): RF = {rf}")

    # GAP doesn't change RF
    print(f"AdaptiveAvgPool2d(1): RF = {rf} (unchanged)")

    print(f"\nFinal receptive field: {rf} x {rf} pixels")
    return rf

def count_parameters(model):
    """Count total parameters in the model"""
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("\nModel Parameters:")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    return total_params

In [8]:
# ================================
# main.py - Main Training Orchestration
# ================================
class get_model:
    def __init__(self,device=None):
        self.device = device if device else self.get_device()
        self.model_obj = self.get_model()
        self.model_config = self.get_config()

    def get_device(self):
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def get_model(self):
        return Net().to(self.device)

    def get_config(self):
        return set_config_v0().setup(self.model_obj)

def main_i(params_check=1):
    logging.info("Setting up for model")
    model = get_model(device=None)
    # Capture printed summary into logs
    import io
    import contextlib
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        summary(model.model_obj, input_size=(3, 32, 32))
        summary_text = buf.getvalue().strip()
    if summary_text:
        logging.info("\n" + summary_text)
    train_test_instance = train_test_model(model.model_obj,
                                          model.device,
                                          model.model_config.data_setup_instance.train_loader,
                                          model.model_config.data_setup_instance.test_loader,
                                          model.model_config.criterion,
                                          model.model_config.optimizer,
                                          model.model_config.scheduler,
                                          model.model_config.epochs)
    if (params_check == 0):
        train_test_instance.run_epoch()
    else:
        pass
    #train_test_instance.plot_results()
    # Capture printed model checks into logs
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        model_checks(model.model_obj)
        checks_text = buf.getvalue().strip()
    if checks_text:
        logging.info("\n" + checks_text)

def main():
    # Initialize logging only in the main process
    setup_logging(log_to_file=True)
    params_check = int(input("Enter 1 for params check only, 0 for full training/testing: "))
    main_i(params_check=params_check)

if __name__ == "__main__":
    main()

KeyboardInterrupt: Interrupted by user

In [10]:
# ================================
# main.py - Main Training Orchestration
# ================================
class get_model:
    def __init__(self,device=None):
        self.device = device if device else self.get_device()
        self.model_obj = self.get_model()
        self.model_config = self.get_config()

    def get_device(self):
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def get_model(self):
        return Net().to(self.device)

    def get_config(self):
        return set_config_v0().setup(self.model_obj)

def main_i(params_check=1):
    logging.info("Setting up for model")
    model = get_model(device=None)
    # Capture printed summary into logs
    import io
    import contextlib
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        summary(model.model_obj, input_size=(3, 32, 32))
        summary_text = buf.getvalue().strip()
    if summary_text:
        logging.info("\n" + summary_text)
    train_test_instance = train_test_model(model.model_obj,
                                          model.device,
                                          model.model_config.data_setup_instance.train_loader,
                                          model.model_config.data_setup_instance.test_loader,
                                          model.model_config.criterion,
                                          model.model_config.optimizer,
                                          model.model_config.scheduler,
                                          model.model_config.epochs)
    if (params_check == 0):
        train_test_instance.run_epoch()
    else:
        pass
    #train_test_instance.plot_results()
    # Capture printed model checks into logs
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        model_checks(model.model_obj)
        checks_text = buf.getvalue().strip()
    if checks_text:
        logging.info("\n" + checks_text)

def main():
    # Initialize logging only in the main process
    setup_logging(log_to_file=True)
    params_check = int(input("Enter 1 for params check only, 0 for full training/testing: "))
    main_i(params_check=params_check)

if __name__ == "__main__":
    main()

Enter 1 for params check only, 0 for full training/testing: 0
2025-10-02 22:28:19,103 - INFO - Setting up for model
2025-10-02 22:28:19,110 - INFO - Model v0 dataloader args: {'batch_size_train': 128, 'batch_size_test': 1000, 'shuffle_train': True, 'shuffle_test': False, 'num_workers': 2, 'pin_memory': True}


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipython-input-2167529188.py:48: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


2025-10-02 22:28:20,683 - INFO - Model v0: OneCycleLR max_lr=0.01 pct_start=0.2 div_factor=10 final_div_factor=100 epochs=15
2025-10-02 22:28:20,683 - INFO - Dataloader arguments: {'batch_size_train': 128, 'batch_size_test': 1000, 'shuffle_train': True, 'shuffle_test': False, 'num_workers': 2, 'pin_memory': True}
2025-10-02 22:28:20,724 - INFO - 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             216
       BatchNorm2d-2            [-1, 8, 32, 32]              16
              ReLU-3            [-1, 8, 32, 32]               0
           Dropout-4            [-1, 8, 32, 32]               0
            Conv2d-5           [-1, 16, 32, 32]           1,152
       BatchNorm2d-6           [-1, 16, 32, 32]              32
              ReLU-7           [-1, 16, 32, 32]               0
           Dropout-8           [-1, 16, 32, 32]               0
          

Train Loss=0.0154 Acc=24.35% LR=0.0033: 100%|██████████| 391/391 [00:22<00:00, 17.74it/s]


2025-10-02 22:28:42,772 - INFO - Epoch 01/15: Train set final results: Average loss: 768.4059, Accuracy: 12176/50000 (24.35%)


...

2025-10-02 22:33:57,069 - INFO - Epoch 14/15:Test set final results: Average loss: 0.6586, Accuracy: 7709/10000 (77.09%)


Train Loss=0.0056 Acc=74.55% LR=0.0000: 100%|██████████| 391/391 [00:22<00:00, 17.24it/s]


2025-10-02 22:34:19,748 - INFO - Epoch 15/15: Train set final results: Average loss: 279.6953, Accuracy: 37274/50000 (74.55%)


Test Loss=0.6558 Accuracy=77.01%: 100%|██████████| 10/10 [00:01<00:00,  5.97it/s]

2025-10-02 22:34:21,426 - INFO - Epoch 15/15:Test set final results: Average loss: 0.6558, Accuracy: 7701/10000 (77.01%)
2025-10-02 22:34:21,431 - INFO - 
2025-10-02 22:34:21,426 - INFO - --- Model Architecture Checks ---
2025-10-02 22:34:21,427 - INFO - Total Parameters: 148,466
2025-10-02 22:34:21,427 - INFO - Trainable Parameters: 148,466

2025-10-02 22:34:21,427 - INFO - Layer-wise Parameter Details (in model order):
2025-10-02 22:34:21,427 - INFO - ----------------------------------------------------------------------------------------------------
2025-10-02 22:34:21,427 - INFO - 
Block: c1 (Sequential)
2025-10-02 22:34:21,428 - INFO -   c1.0 (Conv2d)                                      | Params:    216 | Convolution: 3->8 channels, kernel (3, 3), stride (1, 1), padding (1, 1), groups 1, bias False
2025-10-02 22:34:21,428 - INFO -   c1.1 (BatchNorm2d)                                 | Params:     16 | BatchNorm: 8 features, eps=1e-05, momentum=0.1
2025-10-02 22:34:21,428 - INFO -

In [13]:
# ================================
# main.py - Main Training Orchestration
# ================================
class get_model:
    def __init__(self,device=None):
        self.device = device if device else self.get_device()
        self.model_obj = self.get_model()
        self.model_config = self.get_config()

    def get_device(self):
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def get_model(self):
        return Net().to(self.device)

    def get_config(self):
        return set_config_v0().setup(self.model_obj)

def main_i(params_check=1):
    logging.info("Setting up for model")
    model = get_model(device=None)
    # Capture printed summary into logs
    import io
    import contextlib
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        summary(model.model_obj, input_size=(3, 32, 32))
        summary_text = buf.getvalue().strip()
    if summary_text:
        logging.info("\n" + summary_text)
    train_test_instance = train_test_model(model.model_obj,
                                          model.device,
                                          model.model_config.data_setup_instance.train_loader,
                                          model.model_config.data_setup_instance.test_loader,
                                          model.model_config.criterion,
                                          model.model_config.optimizer,
                                          model.model_config.scheduler,
                                          model.model_config.epochs)
    if (params_check == 0):
        train_test_instance.run_epoch()
    else:
        pass
    #train_test_instance.plot_results()
    # Capture printed model checks into logs
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        model_checks(model.model_obj)
        checks_text = buf.getvalue().strip()
    if checks_text:
        logging.info("\n" + checks_text)

def main():
    # Initialize logging only in the main process
    setup_logging(log_to_file=True)
    params_check = int(input("Enter 1 for params check only, 0 for full training/testing: "))
    main_i(params_check=params_check)

if __name__ == "__main__":
    main()

Enter 1 for params check only, 0 for full training/testing: 0
2025-10-02 22:53:49,846 - INFO - Setting up for model
2025-10-02 22:53:49,857 - INFO - Model v0 dataloader args: {'batch_size_train': 128, 'batch_size_test': 1000, 'shuffle_train': True, 'shuffle_test': False, 'num_workers': 2, 'pin_memory': True}


/tmp/ipython-input-2167529188.py:48: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


2025-10-02 22:53:51,445 - INFO - Model v0: OneCycleLR max_lr=0.01 pct_start=0.2 div_factor=10 final_div_factor=100 epochs=15
2025-10-02 22:53:51,446 - INFO - Dataloader arguments: {'batch_size_train': 128, 'batch_size_test': 1000, 'shuffle_train': True, 'shuffle_test': False, 'num_workers': 2, 'pin_memory': True}
2025-10-02 22:53:51,452 - INFO - 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             216
       BatchNorm2d-2            [-1, 8, 32, 32]              16
              ReLU-3            [-1, 8, 32, 32]               0
           Dropout-4            [-1, 8, 32, 32]               0
            Conv2d-5           [-1, 16, 32, 32]           1,152
       BatchNorm2d-6           [-1, 16, 32, 32]              32
              ReLU-7           [-1, 16, 32, 32]               0
           Dropout-8           [-1, 16, 32, 32]               0
          

Train Loss=0.0156 Acc=23.10% LR=0.0033: 100%|██████████| 391/391 [00:21<00:00, 18.42it/s]


2025-10-02 22:54:12,677 - INFO - Epoch 01/15: Train set final results: Average loss: 781.3326, Accuracy: 11551/50000 (23.10%)


...

2025-10-02 22:59:27,999 - INFO - Epoch 14/15:Test set final results: Average loss: 0.6624, Accuracy: 7678/10000 (76.78%)


Train Loss=0.0057 Acc=74.68% LR=0.0000: 100%|██████████| 391/391 [00:21<00:00, 17.90it/s]


2025-10-02 22:59:49,844 - INFO - Epoch 15/15: Train set final results: Average loss: 282.9829, Accuracy: 37338/50000 (74.68%)


Test Loss=0.6517 Accuracy=77.33%: 100%|██████████| 10/10 [00:01<00:00,  5.76it/s]

2025-10-02 22:59:51,584 - INFO - Epoch 15/15:Test set final results: Average loss: 0.6517, Accuracy: 7733/10000 (77.33%)
2025-10-02 22:59:51,590 - INFO - 
2025-10-02 22:59:51,584 - INFO - --- Model Architecture Checks ---
2025-10-02 22:59:51,585 - INFO - Total Parameters: 157,778
2025-10-02 22:59:51,585 - INFO - Trainable Parameters: 157,778

2025-10-02 22:59:51,585 - INFO - Layer-wise Parameter Details (in model order):
2025-10-02 22:59:51,585 - INFO - ----------------------------------------------------------------------------------------------------
2025-10-02 22:59:51,585 - INFO - 
Block: c1 (Sequential)
2025-10-02 22:59:51,585 - INFO -   c1.0 (Conv2d)                                      | Params:    216 | Convolution: 3->8 channels, kernel (3, 3), stride (1, 1), padding (1, 1), groups 1, bias False
2025-10-02 22:59:51,586 - INFO -   c1.1 (BatchNorm2d)                                 | Params:     16 | BatchNorm: 8 features, eps=1e-05, momentum=0.1
2025-10-02 22:59:51,586 - INFO -

In [9]:
# ================================
# main.py - Main Training Orchestration
# ================================
class get_model:
    def __init__(self,device=None):
        self.device = device if device else self.get_device()
        self.model_obj = self.get_model()
        self.model_config = self.get_config()

    def get_device(self):
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def get_model(self):
        return Net().to(self.device)

    def get_config(self):
        return set_config_v0().setup(self.model_obj)

def main_i(params_check=1):
    logging.info("Setting up for model")
    model = get_model(device=None)
    # Capture printed summary into logs
    import io
    import contextlib
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        summary(model.model_obj, input_size=(3, 32, 32))
        summary_text = buf.getvalue().strip()
    if summary_text:
        logging.info("\n" + summary_text)
    train_test_instance = train_test_model(model.model_obj,
                                          model.device,
                                          model.model_config.data_setup_instance.train_loader,
                                          model.model_config.data_setup_instance.test_loader,
                                          model.model_config.criterion,
                                          model.model_config.optimizer,
                                          model.model_config.scheduler,
                                          model.model_config.epochs)
    if (params_check == 0):
        train_test_instance.run_epoch()
    else:
        pass
    #train_test_instance.plot_results()
    # Capture printed model checks into logs
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        model_checks(model.model_obj)
        checks_text = buf.getvalue().strip()
    if checks_text:
        logging.info("\n" + checks_text)

def main():
    # Initialize logging only in the main process
    setup_logging(log_to_file=True)
    params_check = int(input("Enter 1 for params check only, 0 for full training/testing: "))
    main_i(params_check=params_check)

if __name__ == "__main__":
    main()

Enter 1 for params check only, 0 for full training/testing: 0
2025-10-03 00:10:14,510 - INFO - Setting up for model
2025-10-03 00:10:14,855 - INFO - Model v0 dataloader args: {'batch_size_train': 32, 'batch_size_test': 1000, 'shuffle_train': True, 'shuffle_test': False, 'num_workers': 2, 'pin_memory': True}


/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipython-input-2167529188.py:48: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(
100%|██████████| 170M/170M [01:03<00:00, 2.67MB/s]


2025-10-03 00:11:22,027 - INFO - Model v0: OneCycleLR max_lr=0.01 pct_start=0.2 div_factor=10 final_div_factor=100 epochs=35
2025-10-03 00:11:22,028 - INFO - Dataloader arguments: {'batch_size_train': 32, 'batch_size_test': 1000, 'shuffle_train': True, 'shuffle_test': False, 'num_workers': 2, 'pin_memory': True}
2025-10-03 00:11:23,006 - INFO - 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             216
       BatchNorm2d-2            [-1, 8, 32, 32]              16
              ReLU-3            [-1, 8, 32, 32]               0
           Dropout-4            [-1, 8, 32, 32]               0
            Conv2d-5           [-1, 16, 32, 32]           1,152
       BatchNorm2d-6           [-1, 16, 32, 32]              32
              ReLU-7           [-1, 16, 32, 32]               0
           Dropout-8           [-1, 16, 32, 32]               0
           

Train Loss=0.0557 Acc=31.64% LR=0.0014: 100%|██████████| 1563/1563 [00:35<00:00, 44.28it/s]


2025-10-03 00:11:58,307 - INFO - Epoch 01/35: Train set final results: Average loss: 2787.2644, Accuracy: 15822/50000 (31.64%)


...

2025-10-03 00:32:31,736 - INFO - Epoch 34/35:Test set final results: Average loss: 0.4455, Accuracy: 8490/10000 (84.90%)


Train Loss=0.0153 Acc=83.04% LR=0.0000: 100%|██████████| 1563/1563 [00:35<00:00, 44.54it/s]


2025-10-03 00:33:06,831 - INFO - Epoch 35/35: Train set final results: Average loss: 766.9637, Accuracy: 41519/50000 (83.04%)


Test Loss=0.4484 Accuracy=85.09%: 100%|██████████| 10/10 [00:01<00:00,  6.07it/s]

2025-10-03 00:33:08,482 - INFO - Epoch 35/35:Test set final results: Average loss: 0.4484, Accuracy: 8509/10000 (85.09%)
2025-10-03 00:33:08,488 - INFO - 
2025-10-03 00:33:08,483 - INFO - --- Model Architecture Checks ---
2025-10-03 00:33:08,483 - INFO - Total Parameters: 157,778
2025-10-03 00:33:08,483 - INFO - Trainable Parameters: 157,778

2025-10-03 00:33:08,483 - INFO - Layer-wise Parameter Details (in model order):
2025-10-03 00:33:08,484 - INFO - ----------------------------------------------------------------------------------------------------
2025-10-03 00:33:08,484 - INFO - 
Block: c1 (Sequential)
2025-10-03 00:33:08,484 - INFO -   c1.0 (Conv2d)                                      | Params:    216 | Convolution: 3->8 channels, kernel (3, 3), stride (1, 1), padding (1, 1), groups 1, bias False
2025-10-03 00:33:08,484 - INFO -   c1.1 (BatchNorm2d)                                 | Params:     16 | BatchNorm: 8 features, eps=1e-05, momentum=0.1
2025-10-03 00:33:08,484 - INFO -

## Model Architecture Overview

The model implements:
- **C1**: Initial feature extraction (3→8→16→32 channels)
- **C2**: Dilated convolutions (dilation=1,2,4) maintaining 32x32 spatial size
- **C3**: Depthwise Separable Convolution with stride=2 (32→64 channels, 16x16)
- **C4**: Final convolution with stride=2 (64→128 channels, 8x8) + GAP + FC

**Key Features:**
- Receptive field: 94 pixels (>44 requirement)
- Parameters: ~150k (<200k requirement)
- Albumentations: HorizontalFlip, ShiftScaleRotate, CoarseDropout

In [ ]:
# Create and analyze the model
model = Net()
print(f"Model created successfully!")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

# Calculate receptive field
rf = calculate_receptive_field(model)
params = count_parameters(model)

print("
Requirements check:")
print(f"RF > 44: {'✓' if rf > 44 else '✗'} ({rf} > 44)")
print(f"Parameters < 200k: {'✓' if params < 200000 else '✗'} ({params:,} < 200,000)")

In [ ]:
# Model summary
import io
import contextlib

with io.StringIO() as buf, contextlib.redirect_stdout(buf):
    summary(model, input_size=(3, 32, 32))
    summary_text = buf.getvalue()

print("Model Summary:")
print(summary_text)

## Training Setup

Now let's set up the training configuration and data loaders.

In [ ]:
# Setup training
model_wrapper = get_model(device=None)
print("Training setup complete!")
print(f"Device: {model_wrapper.device}")
print(f"Epochs: {model_wrapper.model_config.epochs}")

## Start Training

Choose your training mode:
- Enter 0 for full training/testing
- Enter 1 for parameter check only

In [ ]:
# Choose training mode
params_check = 0  # Set to 0 for full training, 1 for params check only

if params_check == 0:
    print("Starting FULL TRAINING...")
    main_i(params_check=0)
else:
    print("Parameter check only...")
    main_i(params_check=1)

## Results and Analysis

After training completes, you can analyze the results:

1. **Training curves**: Loss and accuracy over epochs
2. **Final accuracy**: Should achieve >85% on CIFAR-10 test set
3. **Model performance**: Check if all requirements are met

## Key Requirements Met:
- ✅ C1C2C3C4 architecture with no MaxPooling
- ✅ Last convolution has stride=2
- ✅ Receptive field > 44 pixels
- ✅ One Depthwise Separable Convolution layer
- ✅ One Dilated Convolution layer
- ✅ Global Average Pooling (GAP)
- ✅ Albumentations data augmentation
- ✅ < 200k parameters
- ✅ Code modularity
- ✅ 85% accuracy target

## Troubleshooting

If you encounter issues:
1. Make sure all dependencies are installed
2. Check that CUDA is available if using GPU
3. Monitor memory usage on Colab
4. Training may take 30-60 minutes depending on hardware

In [ ]:
# Install required packages
!pip install torch torchvision torchsummary numpy matplotlib albumentations tqdm

In [ ]:
# Clone the repository (replace with your actual repo URL)
!git clone https://github.com/your-username/ERA_v4_cifar_10_model_v1_S7.git
%cd ERA_v4_cifar_10_model_v1_S7

## Alternative: Upload Project Files

If you prefer to upload the files directly instead of cloning, upload all the .py files from your project to Colab.

In [ ]:
# If uploading files manually, create the project structure
# Upload all .py files to Colab and run this cell
import os
if not os.path.exists('ERA_v4_cifar_10_model_v1_S7'):
    os.makedirs('ERA_v4_cifar_10_model_v1_S7')
    print("Created project directory. Please upload your .py files.")
else:
    %cd ERA_v4_cifar_10_model_v1_S7

## Model Architecture Overview

The model implements:
- **C1**: Initial feature extraction (3→8→16→32 channels)
- **C2**: Dilated convolutions (dilation=1,2,4) maintaining 32x32 spatial size
- **C3**: Depthwise Separable Convolution with stride=2 (32→64 channels, 16x16)
- **C4**: Final convolution with stride=2 (64→128 channels, 8x8) + GAP + FC

**Key Features:**
- Receptive field: 94 pixels (>44 requirement)
- Parameters: ~150k (<200k requirement)
- Albumentations: HorizontalFlip, ShiftScaleRotate, CoarseDropout

In [ ]:
# Check if all required files are present
import os
required_files = [
    'main.py', 'cifar10model_v0.py', 'data_setup.py', 'data_visual.py',
    'train_test.py', 'logger_setup.py', 'summarizer.py'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    print(f"Missing files: {missing_files}")
    print("Please upload the missing files to continue.")
else:
    print("All required files are present!")

In [ ]:
# Import and check model
try:
    from cifar10model_v0 import Net
    import torch

    # Create model
    model = Net()
    print(f"Model created successfully!")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Check receptive field calculation
    print("\nRunning receptive field calculation...")

except ImportError as e:
    print(f"Import error: {e}")
    print("Please ensure all Python files are uploaded.")

In [ ]:
# Run receptive field calculator
try:
    from receptive_field_calculator import calculate_receptive_field, count_parameters

    rf = calculate_receptive_field(model)
    params = count_parameters(model)

    print("
Requirements check:")
    print(f"RF > 44: {'✓' if rf > 44 else '✗'} ({rf} > 44)")
    print(f"Parameters < 200k: {'✓' if params < 200000 else '✗'} ({params:,} < 200,000)")

except ImportError:
    print("Receptive field calculator not found. Creating a simple version...")

    # Simple parameter count
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")
    print(f"Parameters < 200k: {'✓' if total_params < 200000 else '✗'}")

## Data Visualization

Let's visualize the CIFAR-10 dataset and understand the data distribution.

In [ ]:
# Visualize CIFAR-10 data
try:
    from data_visual import data_visual
    import matplotlib.pyplot as plt

    # Create data visualizer
    visualizer = data_visual(dataset_name='cifar10')

    # Show sample images
    visualizer.show_sample_images()

    # Show class distribution
    visualizer.show_class_distribution()

except ImportError as e:
    print(f"Data visualization error: {e}")
    print("Continuing without visualization...")

## Model Summary

Let's examine the model architecture and parameters.

In [ ]:
# Model summary
try:
    from summarizer import summary
    import io
    import contextlib

    # Capture model summary
    with io.StringIO() as buf, contextlib.redirect_stdout(buf):
        summary(model, input_size=(3, 32, 32))
        summary_text = buf.getvalue()

    print("Model Summary:")
    print(summary_text)

except ImportError as e:
    print(f"Summary error: {e}")
    print("Model structure:")
    print(model)

## Training Setup

Now let's set up the training configuration and data loaders.

In [ ]:
# Setup training
try:
    from main import get_model

    # Create model and configuration
    model_wrapper = get_model(model_version=0)
    print("Training setup complete!")
    print(f"Device: {model_wrapper.device}")
    print(f"Epochs: {model_wrapper.model_config.epochs}")

except Exception as e:
    print(f"Setup error: {e}")
    print("Manual setup...")

    # Manual setup if main.py fails
    from cifar10model_v0 import Net, set_config_v0
    from data_setup import DataSetup

    model = Net()
    config = set_config_v0()
    config.setup(model)
    print("Manual setup complete!")

## Start Training

Finally, let's start the training process. This will train the model for the specified number of epochs.

In [ ]:
# Start training
try:
    from train_test import train_test_model

    # Run training
    print("Starting training...")
    train_test_model(
        model_wrapper.model_obj,
        model_wrapper.device,
        model_wrapper.model_config.data_setup_instance.train_loader,
        model_wrapper.model_config.data_setup_instance.test_loader,
        model_wrapper.model_config.criterion,
        model_wrapper.model_config.optimizer,
        model_wrapper.model_config.scheduler
    )

except Exception as e:
    print(f"Training error: {e}")
    print("Please check the setup and try again.")

## Results and Analysis

After training completes, you can analyze the results:

1. **Training curves**: Loss and accuracy over epochs
2. **Final accuracy**: Should achieve >85% on CIFAR-10 test set
3. **Model performance**: Check if all requirements are met

## Key Requirements Met:
- ✅ C1C2C3C4 architecture with no MaxPooling
- ✅ Last convolution has stride=2
- ✅ Receptive field > 44 pixels
- ✅ One Depthwise Separable Convolution layer
- ✅ One Dilated Convolution layer
- ✅ Global Average Pooling (GAP)
- ✅ Albumentations data augmentation
- ✅ < 200k parameters
- ✅ Code modularity
- ✅ 85% accuracy target

## Troubleshooting

If you encounter issues:
1. Make sure all Python files are uploaded
2. Check that all dependencies are installed
3. Ensure CUDA is available if using GPU
4. Monitor memory usage on Colab